In [ ]:
import keras_tuner as kt

# Define a model-building function for Keras Tuner
def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    
    x = Flatten()(base_model.output)
    x = Dense(hp.Int('units', min_value=64, max_value=512, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x)
    output = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Instantiate Keras Tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    directory='my_dir',
    project_name='breast_cancer_tuning'
)

# Run hyperparameter search
tuner.search(train_generator, validation_data=validation_generator, epochs=20)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Optimal Units: {best_hps.get('units')}")
print(f"Optimal Dropout: {best_hps.get('dropout')}")
print(f"Optimal Learning Rate: {best_hps.get('learning_rate')}")
